# Modèle d'Ising adapté à l'Éconophysique

**Auteurs**: Adam GUEMOUNE, Sidney VERRÉ, Qirui MA, Bolarinwa SOUHOUIN, Yannick TAPSOBA

**Date**: Juillet 2025

Ce notebook present une implementation du modèle d'Ising adapté pour analyse des marchés financiers. Comme les particules qui hésitent entre monter ou descendre dans un champ magnétique, les traders sur marchés financiers ont aussi leurs moments d'indécision collective - mais avec conséquences beaucoup plus coûteuses!

Dans ce projet, nous explorons comment ce modèle de physique statistique peut être utilisé pour comprendre formations des corrélations entre différents actifs financiers.

In [1]:
# Configuration pour assurer que les graphiques ne s'affichent pas en pop-up
import matplotlib
matplotlib.use('Agg')  
import matplotlib.pyplot as plt
plt.ioff()
import os
os.environ['TERM'] = 'dumb'

# Importation des bibliothèques nécessaires
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import os

# Créer dossier pour les images 
output_dir = "ising_output"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

## 1. Théorie du modèle

Dans modèle d'Ising adapté à éconophysique, nous considérons $K$ réseaux carrés de taille $n \times n$, chacun représentant une action financière. Chaque site $(i,j)$ dans réseau $k$ correspond à un trader dont opinion est représentée par un spin $S_{i,j}^{(k)} \in \{-1,+1\}$, où $+1$ représente position d'achat et $-1$ représente position de vente.

L'Hamiltonien local pour chaque site $(i,j)$ du réseau $k$ est défini par:

$$h_{i,j}^{(k)}(t) = -\sum_{i',j'} S_{i',j'}^{(k)}(t) + \alpha S_{i,j}^{(k)}(t)|M^{(k)}(t)| - \sum_{k_1 \neq k} \gamma M^{(k_1)}(t)$$

où:
* Premier terme représente influence des 8 voisins directs
* Deuxième terme quantifie sensibilité à état global de l'action $k$, avec $M^{(k)}(t)$ qui est magnétisation moyenne (sentiment du marché)
* Troisième terme modélise influence des autres actions sur action $k$, avec $\gamma$ le coefficient de couplage


La dynamique des spins suit une mise à jour probabiliste:

$$S_{i,j}^{(k)}(t+1) = \begin{cases} 
+1 & \text{avec possibilité } p = \frac{e^{-2\beta h_{i,j}^{(k)}(t)}}{1+e^{-2\beta h_{i,j}^{(k)}(t)}} \\
-1 & \text{avec possibilité } 1-p 
\end{cases}$$

## 2. Implémentation des fonctions

Nous implementons maintenant fonctions nécessaires pour simuler notre modèle.

In [2]:
def initialiser_reseau(n, K):
    """
    Initialise aléatoirement un réseau de spins pour K actions
    n: taille du réseau (n x n)
    K: nombre d'actions
    Retourne: S de taille n x n x K avec des valeurs +1 ou -1
    """
    return 2 * (np.random.rand(n, n, K) > 0.5).astype(int) - 1

def calculer_magnetisation(S):
    """
    Calcule la magnétisation moyenne pour chaque action
    S: réseau de spins (n x n x K)
    Retourne: vecteur M de taille K
    """
    n, _, K = S.shape
    M = np.zeros(K)
    
    for k in range(K):
        M[k] = np.mean(S[:, :, k])
    
    return M

In [3]:
def calculer_couches_hamiltonien(S, k, alpha, gamma, M):
    """
    Calcule les trois couches de l'hamiltonien pour l'action k
    S: réseau de spins (n x n x K)
    k: indice de l'action
    alpha: paramètre de sensibilité à l'état global
    gamma: coefficient de couplage entre actions
    M: magnétisation de chaque action
    Retourne: trois matrices correspondant aux trois termes de l'hamiltonien
    """
    n, _, K = S.shape
    terme1_layer = np.zeros((n, n))
    terme2_layer = np.zeros((n, n))
    terme3_layer = np.zeros((n, n))
    
    # Précalculer le terme 3 (constant pour tous les sites d'une même action)
    terme3_val = 0
    for k1 in range(K):
        if k1 != k:
            terme3_val -= gamma * M[k1]
    
    terme3_layer.fill(terme3_val)
    
    # Calculer termes 1 et 2 pour chaque site
    for i in range(n):
        for j in range(n):
            # Calcul des indices des 8 voisins avec conditions périodiques
            voisins_i = [(i-1)%n, i, (i+1)%n, (i-1)%n, (i+1)%n, (i-1)%n, i, (i+1)%n]
            voisins_j = [(j-1)%n, (j-1)%n, (j-1)%n, j, j, (j+1)%n, (j+1)%n, (j+1)%n]
            
            # Calcul de la somme des spins des voisins
            somme_voisins = 0
            for v in range(8):
                somme_voisins += S[voisins_i[v], voisins_j[v], k]
            
            # Terme 1: Influence des voisins
            terme1_layer[i, j] = -somme_voisins
            
            # Terme 2: Sensibilité à l'état global du stock k
            terme2_layer[i, j] = alpha * S[i, j, k] * abs(M[k])
    
    return terme1_layer, terme2_layer, terme3_layer

def calculer_hamiltonien_total(terme1_layer, terme2_layer, terme3_layer):
    """
    Calcule l'hamiltonien total à partir des trois couches
    """
    return terme1_layer + terme2_layer + terme3_layer

In [4]:
def mettre_a_jour_spins(S, beta, alpha, gamma, M):
    """
    Met à jour tous les spins selon la dynamique du modèle
    S: réseau de spins actuel (n x n x K)
    beta: paramètre beta = 1/(k_B*T)
    alpha: paramètre de sensibilité à l'état global
    gamma: coefficient de couplage entre actions
    M: magnétisation de chaque action
    Retourne: réseau de spins mis à jour
    """
    n, _, K = S.shape
    S_next = S.copy()
    
    for k in range(K):
        # Pré-calculer les couches de l'hamiltonien pour l'action k
        terme1_layer, terme2_layer, terme3_layer = calculer_couches_hamiltonien(S, k, alpha, gamma, M)
        hamiltonien_total = calculer_hamiltonien_total(terme1_layer, terme2_layer, terme3_layer)
        
        for i in range(n):
            for j in range(n):
                # Calculer l'hamiltonien local
                h_local = hamiltonien_total[i, j]
                
                # Calculer la probabilité de flip
                p = np.exp(-2 * beta * h_local) / (1 + np.exp(-2 * beta * h_local))
                
                # Mettre à jour le spin avec probabilité p
                if np.random.rand() < p:
                    S_next[i, j, k] = 1
                else:
                    S_next[i, j, k] = -1
    
    return S_next

In [5]:
def visualiser_reseau(S, k, ax, titre):
    """
    Visualise le réseau de spins pour l'action k
    """
    # Créer une colormap personnalisée: bleu pour -1, rouge pour +1
    cmap = LinearSegmentedColormap.from_list('custom_cmap', ['blue', 'red'], N=2)
    
    im = ax.imshow(S[:, :, k], cmap=cmap, vmin=-1, vmax=1)
    ax.set_title(titre)
    ax.set_xlabel('j')
    ax.set_ylabel('i')
    return im

def visualiser_hamiltonien(terme1, terme2, terme3, hamiltonien_total, k, fig_size=(16, 4)):
    """
    Visualise les trois couches de l'hamiltonien et l'hamiltonien total
    """
    fig, axes = plt.subplots(1, 4, figsize=fig_size)
    
    # Normaliser pour une meilleure visualisation
    vmin = min(terme1.min(), terme2.min(), terme3.min(), hamiltonien_total.min())
    vmax = max(terme1.max(), terme2.max(), terme3.max(), hamiltonien_total.max())
    
    im1 = axes[0].imshow(terme1, cmap='viridis', vmin=vmin, vmax=vmax)
    axes[0].set_title(f'Action {k+1}: Influence des voisins')
    axes[0].set_xlabel('j')
    axes[0].set_ylabel('i')
    plt.colorbar(im1, ax=axes[0])
    
    im2 = axes[1].imshow(terme2, cmap='viridis', vmin=vmin, vmax=vmax)
    axes[1].set_title(f'Action {k+1}: Sensibilité à l\'état global')
    axes[1].set_xlabel('j')
    axes[1].set_ylabel('i')
    plt.colorbar(im2, ax=axes[1])
    
    im3 = axes[2].imshow(terme3, cmap='viridis', vmin=vmin, vmax=vmax)
    axes[2].set_title(f'Action {k+1}: Couplage inter-actions')
    axes[2].set_xlabel('j')
    axes[2].set_ylabel('i')
    plt.colorbar(im3, ax=axes[2])
    
    im4 = axes[3].imshow(hamiltonien_total, cmap='viridis', vmin=vmin, vmax=vmax)
    axes[3].set_title(f'Action {k+1}: Hamiltonien total')
    axes[3].set_xlabel('j')
    axes[3].set_ylabel('i')
    plt.colorbar(im4, ax=axes[3])
    
    plt.tight_layout()
    return fig

## 3. Simulation du cas K=1 (une seule action)

Nous allons d'abord examiner cas simple avec une seule action. Ce cas est important pour comprendre comportement de base du modèle avant d'introduire couplages entre actions.

In [6]:
# Fixer la graine aléatoire pour la reproductibilité
np.random.seed(42)

# Paramètres pour le cas K=1
n = 50          # Taille du réseau
beta = 2        # Paramètre β = 1/(k_B*T)
alpha = 20      # Sensibilité à l'état global
gamma = 0       # Pas de couplage pour K=1
K = 1           # Une seule action
T_steps = 100   # Nombre de mises à jour

print(f"Simulation du modèle d'Ising pour K=1 action")
print(f"Paramètres: n={n}, beta={beta}, alpha={alpha}, gamma={gamma}")

# Initialisation du réseau de spins
S = initialiser_reseau(n, K)

# Calcul de la magnétisation initiale
M = calculer_magnetisation(S)
print(f"Magnétisation initiale: M = {M[0]:.4f}")

# Visualisation du réseau initial
fig_initial, ax_initial = plt.subplots(figsize=(5, 5))
visualiser_reseau(S, 0, ax_initial, 'Réseau initial (t=0)')
plt.tight_layout()
initial_path = f"{output_dir}/K1_reseau_initial.png"
fig_initial.savefig(initial_path, dpi=300, bbox_inches='tight')
plt.close(fig_initial)

# Pour tracer l'évolution de la magnétisation
M_history = [M[0]]

# Simuler l'évolution du système
print("\nSimulation de l'évolution du système...")
for t in range(1, T_steps+1):
    # Mettre à jour les spins
    S = mettre_a_jour_spins(S, beta, alpha, gamma, M)
    
    # Calculer la nouvelle magnétisation
    M = calculer_magnetisation(S)
    M_history.append(M[0])
    
    if t % 10 == 0:
        print(f"Itération {t}/{T_steps}, M = {M[0]:.4f}")

# Visualisation du réseau final
fig_final, ax_final = plt.subplots(figsize=(5, 5))
visualiser_reseau(S, 0, ax_final, f'Réseau final (t={T_steps})')
plt.tight_layout()
final_path = f"{output_dir}/K1_reseau_final.png"
fig_final.savefig(final_path, dpi=300, bbox_inches='tight')
plt.close(fig_final)

# Tracer l'évolution de la magnétisation
fig_mag, ax_mag = plt.subplots(figsize=(10, 5))
ax_mag.plot(range(T_steps+1), M_history, 'b-')
ax_mag.set_title('Évolution de la magnétisation (K=1)')
ax_mag.set_xlabel('Itération (t)')
ax_mag.set_ylabel('Magnétisation M')
ax_mag.grid(True)
ax_mag.axhline(y=0, color='r', linestyle='--')
plt.tight_layout()
mag_path = f"{output_dir}/K1_magnetisation.png"
fig_mag.savefig(mag_path, dpi=300, bbox_inches='tight')
plt.close(fig_mag)

# Analyse de l'hamiltonien final
terme1, terme2, terme3 = calculer_couches_hamiltonien(S, 0, alpha, gamma, M)
hamiltonien_total = calculer_hamiltonien_total(terme1, terme2, terme3)

# Visualisation de l'hamiltonien
fig_ham = visualiser_hamiltonien(terme1, terme2, terme3, hamiltonien_total, 0)
ham_path = f"{output_dir}/K1_hamiltonien.png"
fig_ham.savefig(ham_path, dpi=300, bbox_inches='tight')
plt.close(fig_ham)

print("\nSimulation K=1 terminée.")
print(f"Magnétisation finale: M = {M[0]:.4f}")
print(f"Images sauvegardées dans: {output_dir}")

Simulation du modèle d'Ising pour K=1 action
Paramètres: n=50, beta=2, alpha=20, gamma=0
Magnétisation initiale: M = 0.0096

Simulation de l'évolution du système...
Itération 10/100, M = 0.0440
Itération 20/100, M = 0.0344
Itération 30/100, M = 0.0376
Itération 40/100, M = 0.0368
Itération 50/100, M = 0.0224
Itération 60/100, M = 0.0032
Itération 70/100, M = -0.0256
Itération 80/100, M = -0.0384
Itération 90/100, M = -0.0352
Itération 100/100, M = -0.0376

Simulation K=1 terminée.
Magnétisation finale: M = -0.0376
Images sauvegardées dans: ising_output


### Analyse des résultats pour K=1

Les images sont sauvegardées dans dossier `ising_output`. Nous pouvons voir que système évolue vers un état où majorité des spins s'alignent dans même direction, ce qui correspond à une tendance dominante du marché (soit acheter, soit vendre).

Le terme d'auto-renforcement (sensibilité à état global) devient très important quand magnétisation n'est pas proche de zéro, ce qui peut conduire à phénomènes similaires aux bulles spéculatives dans marchés réels.

## 4. Simulation du cas K=2 (deux actions)

Maintenant, nous allons examiner cas avec deux actions couplées. Ce cas nous permet d'étudier formation des corrélations entre différentes actions par interactions entre traders.

In [ ]:
# Réinitialiser la graine aléatoire
np.random.seed(42)

# Paramètres pour le cas K=2
n = 50          # Taille du réseau
beta = 2        # Paramètre β = 1/(k_B*T)
alpha = 20      # Sensibilité à l'état global
gamma = 0.15    # Coefficient de couplage
K = 2           # Deux actions
T_steps = 100   # Nombre de mises à jour

print(f"Simulation du modèle d'Ising pour K=2 actions")
print(f"Paramètres: n={n}, beta={beta}, alpha={alpha}, gamma={gamma}")

# Initialisation du réseau de spins
S = initialiser_reseau(n, K)

# Calcul de la magnétisation initiale
M = calculer_magnetisation(S)
print("Magnétisations initiales:")
for k in range(K):
    print(f"Action {k+1}: M = {M[k]:.4f}")

# Visualisation des réseaux initiaux
fig_initial, axes_initial = plt.subplots(1, K, figsize=(10, 5))
for k in range(K):
    visualiser_reseau(S, k, axes_initial[k], f'Réseau initial pour action {k+1}')
plt.tight_layout()
initial_path = f"{output_dir}/K2_reseau_initial.png"
fig_initial.savefig(initial_path, dpi=300, bbox_inches='tight')
plt.close(fig_initial)

# Pour tracer l'évolution des magnétisations
M_history = [M.copy()]

# Simuler l'évolution du système
print("\nSimulation de l'évolution du système...")
for t in range(1, T_steps+1):
    # Mettre à jour les spins
    S = mettre_a_jour_spins(S, beta, alpha, gamma, M)
    
    # Calculer la nouvelle magnétisation
    M = calculer_magnetisation(S)
    M_history.append(M.copy())
    
    if t % 10 == 0:
        print(f"Itération {t}/{T_steps}")
        for k in range(K):
            print(f"Action {k+1}: M = {M[k]:.4f}")

# Visualisation des réseaux finaux
fig_final, axes_final = plt.subplots(1, K, figsize=(10, 5))
for k in range(K):
    visualiser_reseau(S, k, axes_final[k], f'Réseau final pour action {k+1}')
plt.tight_layout()
final_path = f"{output_dir}/K2_reseau_final.png"
fig_final.savefig(final_path, dpi=300, bbox_inches='tight')
plt.close(fig_final)

# Tracer l'évolution des magnétisations
M_history = np.array(M_history)
fig_mag, ax_mag = plt.subplots(figsize=(10, 5))
for k in range(K):
    ax_mag.plot(range(T_steps+1), M_history[:, k], label=f'Action {k+1}')
ax_mag.set_title('Évolution des magnétisations (K=2)')
ax_mag.set_xlabel('Itération (t)')
ax_mag.set_ylabel('Magnétisation M')
ax_mag.grid(True)
ax_mag.axhline(y=0, color='r', linestyle='--')
ax_mag.legend()
plt.tight_layout()
mag_path = f"{output_dir}/K2_magnetisation.png"
fig_mag.savefig(mag_path, dpi=300, bbox_inches='tight')
plt.close(fig_mag)

# Calculer la corrélation entre les deux actions
corr = np.corrcoef(M_history[50:, 0], M_history[50:, 1])[0, 1]  # Ignorer les 50 premières itérations
print(f"\nCorrélation entre les magnétisations des deux actions: {corr:.4f}")

# Visualisation des hamiltoniens pour chaque action
for k in range(K):
    terme1, terme2, terme3 = calculer_couches_hamiltonien(S, k, alpha, gamma, M)
    hamiltonien_total = calculer_hamiltonien_total(terme1, terme2, terme3)
    
    fig_ham = visualiser_hamiltonien(terme1, terme2, terme3, hamiltonien_total, k)
    ham_path = f"{output_dir}/K2_hamiltonien_action_{k+1}.png"
    fig_ham.savefig(ham_path, dpi=300, bbox_inches='tight')
    plt.close(fig_ham)

print("\nSimulation K=2 terminée.")
print("Magnétisations finales:")
for k in range(K):
    print(f"Action {k+1}: M = {M[k]:.4f}")
print(f"Images sauvegardées dans: {output_dir}")

### Analyse des résultats pour K=2

Avec introduction du couplage ($\gamma=0.15$), nous observons maintenant comment corrélations entre actions émergent. Les dynamiques des deux actions deviennent partiellement synchronisées à cause de terme de couplage dans Hamiltonien.

Le coefficient de corrélation nous donne une mesure quantitative de cette synchronisation. Dans marchés financiers réels, ces corrélations sont aussi observées, particulièrement pendant périodes de crise où effet de contagion devient important.

## 5. Étude de l'impact du coefficient de couplage $\gamma$

Pour mieux comprendre impact du coefficient de couplage $\gamma$ sur corrélations entre actions, nous allons faire plusieurs simulations avec différentes valeurs de $\gamma$ et analyser comment corrélations changent.

In [ ]:
# Réinitialiser la graine aléatoire
np.random.seed(42)

# Paramètres fixes
n = 30          # Taille réduite pour accélérer les calculs
beta = 2        # Paramètre β = 1/(k_B*T)
alpha = 20      # Sensibilité à l'état global
K = 2           # Deux actions
T_steps = 80    # Nombre de mises à jour
gamma_values = [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3]  # Différentes valeurs de gamma

# Stocker les corrélations pour chaque valeur de gamma
correlations = []

print("Étude de l'impact de gamma sur les corrélations")

for gamma in gamma_values:
    print(f"\nSimulation avec gamma = {gamma}")
    
    # Initialisation du réseau de spins (même initialisation pour toutes les simulations)
    np.random.seed(42)
    S = initialiser_reseau(n, K)
    
    # Calcul de la magnétisation initiale
    M = calculer_magnetisation(S)
    
    # Pour tracer l'évolution des magnétisations
    M_history = [M.copy()]
    
    # Simuler l'évolution du système
    for t in range(1, T_steps+1):
        # Mettre à jour les spins
        S = mettre_a_jour_spins(S, beta, alpha, gamma, M)
        
        # Calculer la nouvelle magnétisation
        M = calculer_magnetisation(S)
        M_history.append(M.copy())
    
    # Calculer la corrélation entre les deux actions (ignorer les 40 premières itérations)
    M_history = np.array(M_history)
    corr = np.corrcoef(M_history[40:, 0], M_history[40:, 1])[0, 1]
    correlations.append(corr)
    
    print(f"Corrélation: {corr:.4f}")

# Tracer la relation entre gamma et corrélation
fig_corr, ax_corr = plt.subplots(figsize=(10, 5))
ax_corr.plot(gamma_values, correlations, 'bo-', linewidth=2)
ax_corr.set_title('Impact de $\gamma$ sur la corrélation entre actions')
ax_corr.set_xlabel('Coefficient de couplage $\gamma$')
ax_corr.set_ylabel('Corrélation')
ax_corr.grid(True)
ax_corr.set_ylim(-1.1, 1.1)
plt.tight_layout()
corr_path = f"{output_dir}/impact_gamma_correlations.png"
fig_corr.savefig(corr_path, dpi=300, bbox_inches='tight')
plt.close(fig_corr)

print("\nÉtude de l'impact de gamma terminée.")
print(f"Résultats:")
for i, gamma in enumerate(gamma_values):
    print(f"Gamma = {gamma}: Corrélation = {correlations[i]:.4f}")
print(f"Graphique sauvegardé: {corr_path}")

### Analyse de l'impact de $\gamma$

Les résultats montrent une relation non-linéaire entre coefficient de couplage $\gamma$ et corrélations entre actions. Cette non-linéarité est particulièrement intéressante parce qu'elle montre que augmentation du couplage entre actions ne conduit pas nécessairement à une augmentation monotone des corrélations, contrairement à ce qu'une intuition pourrait suggérer.

On peut identifier différents régimes de couplage:
* $\gamma \approx 0$: Couplage faible, corrélations principalement dues au hasard
* $0 < \gamma < 0.15$: Couplage modéré, émergence de corrélations structurées
* $\gamma > 0.15$: Couplage fort, synchronisation partielle des actions

Ces résultats ont des implications importantes pour comprendre interconnexion croissante des marchés financiers et ses effets potentiellement contre-intuitifs sur structure des corrélations.

## 6. Limites et extensions possibles

Notre implémentation actuelle a plusieurs limitations qui pourraient être adressées dans extensions futures:

1. **Simplification des marchés réels**: Notre modèle ignore nombreux aspects des marchés réels, comme hétérogénéité des agents ou présence d'informations exogènes.

2. **Paramètres constants**: Les paramètres $\alpha$, $\beta$ et $\gamma$ sont constants, alors qu'ils pourraient varier dans marchés réels selon conditions économiques ou sentiment des investisseurs.

3. **Topologie du réseau**: Nous utilisons réseau carré régulier, tandis que réseaux sociaux réels ont souvent topologie plus complexe (réseaux small-world ou scale-free).

4. **Nombre limité d'actions**: Nous avons principalement étudié cas avec K=1 et K=2 actions, mais marchés réels contiennent beaucoup plus d'actifs avec structures de corrélation complexes.

Extensions possibles incluent:
- Introduction d'hétérogénéité parmi agents
- Ajout d'informations exogènes affectant marché
- Utilisation de topologies de réseau plus réalistes
- Calibration du modèle avec données réelles de marché
- Étude de cas avec K=5 et K=100 actions pour analyser émergence de clusters de corrélation

## 7. Conclusion

Notre étude du modèle d'Ising adapté à l'Éconophysique nous a permis d'analyser en détail mécanismes de formation des corrélations entre actifs financiers et impact des différentes composantes de l'Hamiltonien sur dynamique du système.

Les résultats montrent que coefficient de couplage $\gamma$ influence corrélations de manière non-monotone, suggérant relation complexe entre interconnexion des marchés et structure des corrélations. La décomposition de l'Hamiltonien révèle que, bien que influence des voisins ait amplitude la plus grande, termes de sensibilité globale et couplage inter-actions jouent rôles cruciaux dans synchronisation des actions et formation des clusters.

Ces insights contribuent à meilleure compréhension des phénomènes collectifs sur marchés financiers, en particulier mécanismes de contagion et formation des bulles spéculatives. Ils pourraient aider à développer meilleurs outils pour gestion du risque systémique et prévision des crises financières.

Pour développements futurs, il serait intéressant d'étendre l'analyse à plus grand nombre d'actions et d'introduire plus de réalisme dans modèle en considérant hétérogénéité des agents et informations exogènes.